In [1]:
from ipyleaflet import Map, Marker,Polyline, FullScreenControl, WidgetControl, MarkerCluster, CircleMarker, basemaps, AntPath
from ipywidgets import IntSlider,jslink
import gpxpy
import geopy.distance as distance
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.layouts import row, column
from functools import reduce
from chooser import get_gpx, choose_map

output_notebook()

Loading BokehJS ...

Les cartes qu'on va utiliser :

In [2]:
maps={"OpenStreetMap":basemaps.OpenStreetMap.Mapnik,"TopoMap":basemaps.OpenTopoMap,\
      "HikeBike":basemaps.HikeBike,"MtbMap":basemaps.MtbMap}
name_map,usedmap= choose_map(maps)
print("carte:",name_map)

Cartes disponibles :
0 :  OpenStreetMap
1 :  TopoMap
2 :  HikeBike
3 :  MtbMap
Carte: rang ? (0 par défaut) :1
carte: TopoMap


Le fichier des traces gps :

In [3]:
filegpx= get_gpx()

0 :  ./2021-01-26_10-38-47.gpx
1 :  ./2020-05-11_11-36-45.gpx
2 :  ./2020-05-07_13-49-05.gpx
3 :  ./2020-05-06_12-59-27.gpx
4 :  ./2020-05-05_14-30-13.gpx
5 :  ./2020-05-04_14-23-26.gpx
6 :  ./2020-05-03_14-31-08.gpx
7 :  ./2020-04-29_13-47-10.gpx
8 :  ./2020-04-28_13-27-38.gpx
Fichier.gpx: rang ? 0


On lit le fichier des positions gps, et on récupère les positions sous forme d'une liste de couples (latitude,longitude) :

In [4]:
gpx_file= open(filegpx,"r")
gpx = gpxpy.parse(gpx_file)
points=gpx.tracks[0].segments[0].points

In [5]:
date_debut= gpx.get_time_bounds().start_time.astimezone().strftime("le %d %m %Y à %H heures %M minutes %S secondes")
date_fin= gpx.get_time_bounds().end_time.astimezone().strftime("le %d %m %Y à %H heures %M minutes %S secondes")
print("\nHeure de début : ",date_debut+".","\nHeure de fin :",date_fin+".\n")


Heure de début :  le 26 01 2021 à 10 heures 39 minutes 27 secondes. 
Heure de fin : le 26 01 2021 à 15 heures 12 minutes 51 secondes.



Il faut centrer la carte. Pour cela on calcule la moyenne des latitudes et des longitudes:

In [6]:
l=[(p.latitude,p.longitude) for p in points]
c=[sum(x) for x in zip(*l)]
center=(c[0]/len(l),c[1]/len(l))# le "centre" (la moyenne).

Et on crée la carte, centrée en "center". Le facteur de zoom initial est un peu pifométrique: 

In [7]:
m = Map(basemap=usedmap,center=center, zoom=15,scroll_wheel_zoom=True)

On ajoute une petite tirette pour le zoom:

zoom_slider = IntSlider(description='Zoom:', min=12, max=30, value=15)
jslink((zoom_slider, 'value'), (m, 'zoom'))
widget_control1 = WidgetControl(widget=zoom_slider, position='topright')
m.add_control(widget_control1)

Bien. Maintenant on ajoute la trajectoire, qu'on a déja calculée. On ajoute aussi un bouton "Plein écran".

In [8]:
type_line= "Ant" # type de ligne pour la trajectoire (toute autre avaleur que "Ant" produit une "PolyLine") 

In [9]:
if type_line == "Ant":
    line= AntPath(
        locations= l,
        dash_array=[1, 10],
        delay=2000,
        color='red',
        pulse_color='black'
    )
else:
    line = Polyline(
        locations=l,
        color="red" ,
        fill=False, weight=3
    )

m.add_control(FullScreenControl())
m.add_layer(line)

### Quelques calculs: ###

- Les distances entre les points successifs. C'est un service fourni par geopy (géodésiques sur l'ellipsoïde terrestre).
- La distance globale porcourue.
- Un "marker" tous les "delta" mêtres. 

In [10]:
delta=1000. # 1 km.

In [11]:
distance_parcourue=0.0
dists=[]
next = 0.0
marks=[]
pold=points[0]
for point in points:
    d= point.distance_3d(pold)
    distance_parcourue+= d
    dists.append(distance_parcourue)
    pold= point
    if distance_parcourue >= next:
        next += delta
        marks.append(Marker(location=(point.latitude,point.longitude)))

Des marqueurs pour le début et la fin du parcours :

In [12]:
circle_marker1 = CircleMarker(
    location = (points[-1].latitude,points[-1].longitude),
    radius = 7,color = "red",fill_color = "white")
circle_marker2 = CircleMarker(
    location = (points[0].latitude,points[0].longitude),
    radius = 7,color = "green",fill_color = "green")
marks+=[circle_marker1,circle_marker2]

Placer les marqueurs sur la carte:

In [13]:
marker_cluster = MarkerCluster(
    markers=marks
)
m.add_layer(marker_cluster);

In [14]:
print("Distance totale parcourue :",distance_parcourue,"mêtres.")

Distance totale parcourue : 13850.028056221625 mêtres.


### La carte : ###

_(zoomez éventuellement pour voir tous les marqueurs)._

In [15]:
m

Map(center=[45.3749239173077, 6.493195624260355], controls=(ZoomControl(options=['position', 'zoom_in_text', '…

### L'altitude au cours du parcours : ###

In [16]:
p = figure(title="Altitude /distance", x_axis_label='Distance parcourue', y_axis_label='altitude',
           width=800,height=300)
p.line(dists,[point.elevation for point in points] , line_width=2)
show(p)

### Altitude en fonction du temps :

In [17]:
p = figure(title="Altitude / temps", x_axis_label='Temps (heures)', y_axis_label='altitude',
           width=800,height=300)
heuredep= points[0].time
p.line([(point.time-heuredep).seconds/3600. for point in points],[point.elevation for point in points] , line_width=2)
show(p)

#### Cumul des montées et des descentes : ####

Attention le gps n'est pas très précis pour les altitudes !

In [18]:
z=[x[1]-x[0] for x in zip([point.elevation for point in points][1:],[point.elevation for point in points][:-1])]
up= reduce(lambda a,b: a+max(b,0),z)
down= reduce(lambda a,b: a+max(-b,0),z)

In [19]:
print("montée :",up,", descente :",down,"(mêtres).")

montée : 1021.9300000000005 , descente : 1021.2300000000005 (mêtres).


### Vitesse (en km/h) en fonction du temps (en secondes) : ###

In [20]:
start= points[0].time
z=[((x[0]-x[1]).total_seconds(),x[1]) for x in zip([point.time for point in points][1:], \
                                                   [point.time for point in points][:-1])]
d=[x[1].distance_3d(x[0]) for x in zip(points[1:],points[:-1])]
vt=[(x[0]/x[1][0],(x[1][1]-start).total_seconds())  for x in zip(d,z) if x[1][0]>0]

In [21]:
pv = figure(title="Vitesse / temps", x_axis_label='temps', y_axis_label='Vitesse (km/h)',width=800,height=300)
pv.line([v[1] for v in vt],[v[0]*3.6 for v in vt],  line_width=2)
show(pv)

### Vitesse moyenne : ###

In [22]:
print("%8.2f"% (3.6*distance_parcourue/(points[-1].time - points[0].time).total_seconds()),"km/h.")

    3.04 km/h.


Vitesse en fonction de la déclivité

In [23]:
dec0= [x[1]-x[0] for x in zip([point.elevation for point in points[1:]],[point.elevation for point in points[:-1]])]
dec = [x[0] for x in zip(dec0,z) if x[1][0]>0]
pdec= figure(title="Vitesse / déclivité ", x_axis_label='declivité', y_axis_label='Vitesse (km/h)',width=800,height=300)
pdec.circle(dec,[3.6*x[0] for x in vt],size=6, color="red", alpha=0.5)
show(pdec)